In [2]:
import sys
sys.path.insert(0, '/home/rilab/catkin_ws/src/zavis_ros/osod')

import cv2
import torch
import copy
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

from det.loader import load_det
from det.matcher import matcher
from det.postprocess import post_process,plot,VOC_CLASS_NAMES,CLASS_NAME_NEW

In [3]:
predictor = load_det()
clip = matcher(threshold=30)


/home/rilab/catkin_ws/src/zavis_ros/osod/model/rpn/rpn.py:43: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(layers.weight,std=0.01)
/home/rilab/catkin_ws/src/zavis_ros/osod/model/rpn/rpn.py:44: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(layers.bias,0)


In [9]:
img_files = ['../examples/book_2.jpg','../examples/alarm_clock_1.jpg','../examples/coffee_cup_1.jpg']#,'../examples/food_3.jpg']
NAME_SETS = [['book','red book','trash can'],['redbull','alarmclock','glasses'],['cup','ipod','book']]#,['orange juice','mug cup','cake']]

VOC_COLORS = sns.color_palette('hls', len(CLASS_NAME_NEW)+1)
CLIP_COLORS = sns.color_palette('Set2', len(NAME_SETS)*len(NAME_SETS[0]))

In [12]:
fig = plt.figure(figsize=(15,7))
for i, (img_file, NAME_SET) in enumerate(zip(img_files,NAME_SETS)):
    print(i)
    img = cv2.imread(img_file)


    pred = predictor(img)
    pred = pred['instances']._fields

    pred_boxes = pred['pred_boxes']
    scores = pred['scores']
    pred_classes = pred['pred_classes']
    # print(pred_classes)
    index = torch.where(scores>0.3)[0]

    top_pred_boxes = pred_boxes[index]
    top_pred_classes = pred_classes[index]


    osod_image = copy.deepcopy(img)
    osod_image = cv2.cvtColor(osod_image, cv2.COLOR_RGB2BGR)
    plt.subplot(len(img_files),len(NAME_SET)+1,(len(NAME_SET)+1)*i+1)
    # plt.title("OpenSet")
    plt.imshow(osod_image)
    plt.axis('off')
    for bbox,label in zip(top_pred_boxes,top_pred_classes):
        if label==20:
            color = (0,255,255)
        else:
            color = (255,0,0)
        x1 = int(bbox[0])
        y1 = int(bbox[1])
        x2 = int(bbox[2])
        y2 = int(bbox[3])
        # cv2.rectangle(osod_image, (int(bbox[0]), int(bbox[1])), 
        #                     (int(bbox[2]),int(bbox[3])), color, 5)
        # cv2.putText(osod_image, CLASS_NAME_NEW[int(label)], 
        #                         (int(bbox[0]), int(bbox[1])),
        #                 cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
        plt.gca().add_patch(Rectangle((x1, y1), x2 - x1, y2 - y1,
                                    linewidth=2, edgecolor=VOC_COLORS[label], facecolor='none'))
        plt.text(x1 + 3, y1 + 3, CLASS_NAME_NEW[label],
                bbox=dict(facecolor=VOC_COLORS[label], alpha=0.5), fontsize=15, color='k')
    

    for e,NAME in enumerate(NAME_SET):
        clip.tokenize(NAME)
        clip.thres = 30
        color = (255,100,0)
        mask = top_pred_classes == 20
        score, match_box, _ = clip.matching_score(img,top_pred_boxes[mask])

        clip_image = copy.deepcopy(img)
        clip_image = cv2.cvtColor(clip_image, cv2.COLOR_RGB2BGR)
        plt.subplot(len(img_files),len(NAME_SET)+1,(len(NAME_SET)+1)*i+2+e)
        # plt.title(NAME)
        plt.imshow(clip_image)
        plt.axis('off')
        for s, bbox in zip(score,match_box):
            x1 = int(bbox[0])
            y1 = int(bbox[1])
            x2 = int(bbox[2])
            y2 = int(bbox[3])
            plt.gca().add_patch(Rectangle((x1, y1), x2 - x1, y2 - y1,
                                        linewidth=2, edgecolor=CLIP_COLORS[e+3*i], facecolor='none'))
            plt.text(x1 + 3, y1 + 3, NAME,
                bbox=dict(facecolor=CLIP_COLORS[e+3*i], alpha=0.5), fontsize=15, color='k')
        

plt.tight_layout()
fig.patch.set_visible(False)
plt.show()

0
1
2
